## Setup

In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.34 🚀 Python-3.10.0 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
Setup complete ✅ (16 CPUs, 11.4 GB RAM, 434.6/476.1 GB disk)


In [2]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [3]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [1]

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


## Utils

In [4]:
from utils import *
import torch
import os
import csv
import numpy as np
from IPython import display


In [5]:
import VARIABLES2
import importlib
importlib.reload(VARIABLES2)
from VARIABLES2 import *


vih = VideoInfoHandler()

def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False, device=torch.device("cuda:0"))[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above 
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = vih.byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = vih.trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=vih.label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # update line counter
    for line_zone in vih.line_zones:
        line_zone.trigger(detections)
    # return frame with box and line annotated result
    for i in range(3):
        annotated_frame = vih.line_zone_annotators[i].annotate(annotated_frame, line_counter=vih.line_zones[i])
    return  annotated_frame


videos_folder = "probando"
videos_folder_path = os.path.join("full_recordings", videos_folder)
data = [["file_name", "in", "out"]]

prev_in, prev_out = 0, 0



with open(os.path.join("results", f"{videos_folder}.csv"), "w", newline="") as csv_output:
  writer = csv.writer(csv_output)
  try:
    for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
      file_path = os.path.join(videos_folder_path, file_name)
      vih.re_init(file_path)
      process_video(
        source_path = file_path,
        target_path = TARGET_DUMMY_VIDEO_PATH,
        callback=callback,
      )
      max_in = max(vih.line_zones, key=lambda x: x.in_count)
      max_out = max(vih.line_zones, key=lambda x: x.out_count)
      
      data.append([file_name, max_in.in_count - prev_in, max_out.out_count - prev_out])
      prev_in = max_in.in_count
      prev_out = max_out.out_count
      
  except KeyError as e:
    print(e)
  finally: 
    writer.writerows(data)
    print(max_in.in_count, max_out.out_count)

  

 Videos remaining:   0%|          | 0/2 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]

0 4
